In [ ]:
import plotly.graph_objects as go
import math
import numpy as np

In [ ]:
def fitness(damage, time_to_attack):
    return (damage**3 - damage)/10 + time_to_attack * 5

In [ ]:
damage = np.linspace(0, 10, 100)
time_to_attack = np.linspace(0, 10, 100)

damage_grid, time_to_attack_grid = np.meshgrid(damage, time_to_attack)
fitness_values = fitness(damage_grid, time_to_attack_grid)

fig = go.Figure(data=[go.Surface(z=fitness_values, x=damage_grid, y=time_to_attack_grid)])

# Set plot labels and title
fig.update_layout(
    title='Fitness Function Surface Plot',
    scene=dict(
        xaxis_title='Damage',
        yaxis_title='Time to Attack',
        zaxis_title='Fitness'
    )
)

# Show the plot
fig.show()

## HILL CLIMBING

In [ ]:
# pick random position
weapon_damage = np.random.randint(0,11)
weapon_tta = np.random.randint(0,11)
weapon_fitness = fitness(weapon_damage, weapon_tta)

def clamp(value, min_value, max_value):
    return max(min_value, min(value, max_value))

while True:
    # check if there is an adjacent better fitness coordinate
    best_fitness = weapon_fitness
    weapon_tuple = (weapon_damage, weapon_tta)
    for d in [-1,0,1]:
        for t in [-1,0,1]:
            candidate_fitness = fitness(clamp(weapon_damage + d, 0, 10), clamp(weapon_tta + t, 0, 10))
            if candidate_fitness > best_fitness:
                best_fitness = candidate_fitness
                weapon_tuple = (clamp(weapon_damage + d, 0, 10), clamp(weapon_tta + t, 0, 10))

    # finish if we have not moved
    if weapon_tuple[0] == weapon_damage and weapon_tuple[1] == weapon_tta:
        break

    # move to best found coordinate
    weapon_damage = weapon_tuple[0]
    weapon_tta = weapon_tuple[1]
    weapon_fitness = fitness(weapon_damage, weapon_tta)

print(f"Best weapon damage={weapon_damage} time_to_attack={weapon_tta}")

In [ ]:
fig = go.Figure(data=[go.Surface(z=fitness_values, x=damage_grid, y=time_to_attack_grid)])

fig.add_trace(go.Scatter3d(
    x=[weapon_damage],
    y=[weapon_tta],
    z=[weapon_fitness],
    mode='markers',
    marker=dict(size=10, color='red', symbol='cross')
))

# Set plot labels and title
fig.update_layout(
    title='Fitness Function Surface Plot',
    scene=dict(
        xaxis_title='Damage',
        yaxis_title='Time to Attack',
        zaxis_title='Fitness'
    )
)

# Show the plot
fig.show()

## Evolution

In [ ]:
population_size = 10
generations = 10
mutation_chance = 0.1
population = []

def generate_weapon(damage, time_to_attack):
    fit = fitness(damage, time_to_attack)
    return {"damage": damage, "time_to_attack": time_to_attack, "fitness": fit}

def cross_over_and_mutate(pop1, pop2):
    dmg = pop1["damage"] if np.random.random() < 0.5 else pop2["damage"]
    tta = pop1["time_to_attack"] if np.random.random() < 0.5 else pop2["time_to_attack"]

    if np.random.random() < 0.1:
        if np.random.random() < 0.5:
            dmg = np.random.randint(0,11)
        else:
            tta = np.random.randint(0,11)

    fit = fitness(dmg, tta)
    return {"damage": dmg, "time_to_attack": tta, "fitness": fit}


for i in range(population_size):
    population.append(generate_weapon(np.random.randint(0,11), np.random.randint(0,11)))

for i in range(generations):
    next_generation = []
    for p in range(population_size):
        random_numbers = np.random.randint(0, population_size, size=2)
        next_generation.append(cross_over_and_mutate(population[random_numbers[0]], population[random_numbers[1]]))

    population.extend(next_generation)
    sorted_population = sorted(population, key=lambda x: x["fitness"])

    population = sorted_population[-1*population_size:]

sorted_population = sorted(population, key=lambda x: x["fitness"])
print(sorted_population[0])

In [ ]:
fig = go.Figure(data=[go.Surface(z=fitness_values, x=damage_grid, y=time_to_attack_grid)])

fig.add_trace(go.Scatter3d(
    x=[sorted_population[0]["damage"]],
    y=[sorted_population[0]["time_to_attack"]],
    z=[sorted_population[0]["fitness"]],
    mode='markers',
    marker=dict(size=10, color='red', symbol='cross')
))

# Set plot labels and title
fig.update_layout(
    title='Fitness Function Surface Plot',
    scene=dict(
        xaxis_title='Damage',
        yaxis_title='Time to Attack',
        zaxis_title='Fitness'
    )
)